In [70]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import HTML
import numpy as np

%matplotlib inline

In [63]:
# get a page to pull the team names from first
page = requests.get('http://www.spotrac.com/nfl/dallas-cowboys/positional/2018/full-cap/')
# Create soup of page
soup = BeautifulSoup(page.text, 'html.parser')

In [118]:
# team list pulldown is the first select
teams = soup.find_all('select')[1]
# the team names and abbreviation are in the options
teams = teams.find_all('option')
# skip first option ("Team") and assign team abbreviation: team name to dict
team_names = {option.text: option['value'] for option in teams[1:]}

In [119]:
# create list for years
years = ['2013', '2014', '2015', '2016', '2017', '2018']

In [121]:
# create blank format string for website crawl
url = 'http://www.sportrac.com/nfl/{}/positional/{}/full-cap/'

In [ ]:
def get_positional(soup):
    # find the positional spending table
    table = soup.find_all('table')[1]
    # get the text
    table_text = table.text
    # remove the newlines and make into list
    table_text = list(filter(None, table.text.split('\n')))
    # reshape and convert to data frame
    table_df = pd.DataFrame(np.reshape(table_text, (10, 5)))
    # header is in the first row
    table_df.columns = table_df.iloc[0]
    # drop the first row and reset index
    table_df = table_df.drop(0).reset_index(drop=True)
    # return the final result
    return table

In [125]:
for year in years:
    for team in team_names.keys():
        page = requests.get(url.format(team, year))
        soup = BeautifulSoup(page.text, 'html.parser')

KeyboardInterrupt: 

In [92]:
table = soup.find_all('table')[1]
table_text = table.text
table_text = list(filter(None, table.text.split('\n')))

table_df = pd.DataFrame(np.reshape(table_text, (10, 5)))

table_df.columns = table_df.iloc[0]
table_df = table_df.drop(0).reset_index(drop=True)
table_df.head()

,Position,Players,Cap Dollars,% of Cap,NFL Rank
0,Quarterback,4,"$26,195,076",19.74%,7
1,Running Back,6,"$4,381,412",3.30%,23
2,Wide Receiver,11,"$22,845,000",17.21%,13
3,Tight End,7,"$15,222,083",11.47%,5
4,Offensive Line,13,"$25,383,623",19.12%,26


In [130]:
table_df.melt()

,0,1,2,3,4,5,6,7,8
0,,,,,,,,,
Position,Quarterback,Running Back,Wide Receiver,Tight End,Offensive Line,Defensive Line,Linebacker,Secondary,Special Teams
Players,4,6,11,7,13,12,11,12,2
Cap Dollars,"$26,195,076","$4,381,412","$22,845,000","$15,222,083","$25,383,623","$21,356,636","$14,015,293","$12,122,579","$4,184,000"
% of Cap,19.74%,3.30%,17.21%,11.47%,19.12%,16.09%,10.56%,9.13%,3.15%
NFL Rank,7,23,13,5,26,21,27,32,23
